# Chess Vision System

This notebook implements a computer vision system for analyzing chess games from video footage. The system can:
- Detect the chess board and pieces
- Track piece movements
- Generate chess notation
- Export games to PGN format

## Setup and Dependencies

In [71]:
# Install required packages
# !pip install ultralytics opencv-python numpy torch

In [72]:
import cv2
import torch
import numpy as np
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional
from datetime import datetime
from ultralytics import YOLO
import os
import chess

print(f"OpenCV version: {cv2.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

OpenCV version: 4.10.0
PyTorch version: 2.2.1
CUDA available: True


## Data Structures

First, let's define our core data structures.

In [73]:
@dataclass
class ChessMove:
    piece: str
    from_square: str
    to_square: str
    is_capture: bool
    is_check: bool
    is_checkmate: bool

## Chess Vision System

This class handles the piece detection model.

In [74]:
class ChessVisionSystem:
    def __init__(self, use_pretrained: bool = False, model_path: str = None):
        """Initialize the chess vision system."""
        if use_pretrained and model_path:
            self.model = YOLO(model_path)
        else:
            # Initialize new model for training
            self.model = YOLO("yolo11x.yaml").load("yolo11x.pt")
        
        # self.piece_classes = [
        #     'white_pawn', 'white_knight', 'white_bishop', 'white_rook', 'white_queen', 'white_king',
        #     'black_pawn', 'black_knight', 'black_bishop', 'black_rook', 'black_queen', 'black_king'
        # ]

        self.piece_classes = [
            'bishop', 'black-bishop', 'black-king', 'black-knight', 'black-pawn', 'black-queen', 'black-rook',
            'white-bishop', 'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook'
        ]
        
        
    def train(self, data_yaml_path: str, epochs: int = 100):
        """Train the model on chess piece dataset."""
        self.model.train(
            data=data_yaml_path,
            epochs=epochs,
            imgsz=640,
            batch=2,
            name='my_best_run',
            patience=20,
            save=True,
            device='0' if torch.cuda.is_available() else 'cpu'
        )
    
    def validate(self, data_path: str):
        """Validate the model performance."""
        metrics = self.model.val(data=data_path)
        return metrics

# Hand Detector

In [75]:
# padding: number of pixels to ignore at the edges of the frame
# max_lower_hue: maximum hue value of the hand (0 - 360)
# min_upper_hue: minimum hue value of the hand (0 - 360) 
# min_lightness: minimum lightness value of the hand (0 - 100)
# max_lightness: maximum lightness value of the hand (0 - 100)
# min_saturation: minimum saturation value of the hand (0 - 100)
# threshold: number of pixels that need to be in the frame to be considered a hand
def hand_is_in_frame(
        image: np.ndarray = None, 
        padding: int = 48, 
        max_lower_hue: int = 16, 
        min_upper_hue: int = 350, 
        min_lightness: int = 40, 
        max_lightness: int = 70,
        min_saturation: int = 15,
        threshold: int = 2500,
) -> bool:
    if image is None:
        return True
    
    height, width, _ = image.shape
    image = image[int(height/2-width/2):int(height/2+width/2), 0:]

    height, width, _ = image.shape
    image = image[padding:height-padding, padding:width-padding]
    h, l, s = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2HLS))

    cv_max_lower_hue = int(max_lower_hue * 255 / 360)
    cv_min_upper_hue = int(min_upper_hue * 255 / 360) 
    h_bool = np.logical_or(h < cv_max_lower_hue, h > cv_min_upper_hue)

    cv_min_lightness = int(min_lightness * 255 / 100 )
    cv_max_lightness = int(max_lightness * 255 / 100 )
    l_bool = np.logical_and(l > cv_min_lightness, l < cv_max_lightness)

    cv_min_saturation = int(min_saturation * 255 / 100)
    s_bool = s > cv_min_saturation

    is_hand = np.logical_and(np.logical_and(h_bool, l_bool), s_bool)    
    count = np.count_nonzero(is_hand)
    segmented = np.zeros_like(image)
    segmented[is_hand] = image[is_hand]
    segmented = cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB)

    return count > threshold, image, segmented, count

## Chess Game Analyzer

This class handles the video analysis and move detection.

In [76]:
class ChessGameAnalyzer:
    def __init__(self, model_path: str, video_path: str):
        """Initialize the chess game analyzer."""
        self.vision_system = ChessVisionSystem(use_pretrained=True, model_path=model_path)
        self.cap = cv2.VideoCapture(video_path)
        
        if not self.cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")
        
        # Initialize video dimensions
        ret, frame = self.cap.read()
        if not ret:
            raise ValueError("Could not read first frame")
        
        self.frame_height, self.frame_width = frame.shape[:2]
        
        # Detect board boundaries and calculate transform matrices
        self.detect_board_boundaries(frame)
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        # State tracking
        self.prev_positions = {}
        self.prev_stable_positions = {}
        self.move_list = []
        self.state = 'STABLE'
        self.stable_frame_count = 0
        self.stable_threshold = 20
        self.confidence_threshold = 0.35
        
    def detect_board_boundaries(self, frame: np.ndarray) -> None:
        """Enhanced chess board detection with precise color ranges."""
        # Convert frame to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Convert RGB to BGR
        black_light = [177, 177, 157][::-1]  # Lightest black
        black_dark = [41, 106, 89][::-1]     # Darkest black
        white_light = [240, 240, 240][::-1]  # Lightest white
        white_dark = [180, 190, 180][::-1]   # Darkest white
        
        # Convert to uint8 numpy arrays with correct shape
        black_light_bgr = np.uint8([[black_light]])
        black_dark_bgr = np.uint8([[black_dark]])
        white_light_bgr = np.uint8([[white_light]])
        white_dark_bgr = np.uint8([[white_dark]])
        
        # Convert to HSV
        black_light_hsv = cv2.cvtColor(black_light_bgr, cv2.COLOR_BGR2HSV)[0][0]
        black_dark_hsv = cv2.cvtColor(black_dark_bgr, cv2.COLOR_BGR2HSV)[0][0]
        white_light_hsv = cv2.cvtColor(white_light_bgr, cv2.COLOR_BGR2HSV)[0][0]
        white_dark_hsv = cv2.cvtColor(white_dark_bgr, cv2.COLOR_BGR2HSV)[0][0]
        
        # print(f"Black square HSV ranges: Light={black_light_hsv}, Dark={black_dark_hsv}")
        # print(f"White square HSV ranges: Light={white_light_hsv}, Dark={white_dark_hsv}")
        
        # Create HSV ranges making sure all arrays are uint8
        lower_black = np.array([
            min(black_light_hsv[0], black_dark_hsv[0]),
            min(black_light_hsv[1], black_dark_hsv[1]),
            min(black_light_hsv[2], black_dark_hsv[2])
        ], dtype=np.uint8)
        
        upper_black = np.array([
            np.uint8(max(black_light_hsv[0], black_dark_hsv[0]) + 10),
            np.uint8(max(black_light_hsv[1], black_dark_hsv[1]) + 30),
            np.uint8(max(black_light_hsv[2], black_dark_hsv[2]) + 30)
        ], dtype=np.uint8)
        
        lower_white = np.array([
            min(white_light_hsv[0], white_dark_hsv[0]),
            min(white_light_hsv[1], white_dark_hsv[1]),
            min(white_light_hsv[2], white_dark_hsv[2])
        ], dtype=np.uint8)
        
        upper_white = np.array([
            np.uint8(max(white_light_hsv[0], white_dark_hsv[0]) + 10),
            np.uint8(max(white_light_hsv[1], white_dark_hsv[1]) + 30),
            np.uint8(255)  # Ensure this is uint8
        ], dtype=np.uint8)
        
        # print(f"Upper black: {upper_black}")
        # print(f"Lower black: {lower_black}")

        # print(f"Upper white: {upper_white}")
        # print(f"Lower white: {lower_white}")

        # print(hsv.dtype)
        # print(upper_black.dtype)
        
        # Create masks
        black_mask = cv2.inRange(hsv, lower_black, upper_black)
        white_mask = cv2.inRange(hsv, lower_white, upper_white)
        
        # Combine masks
        board_mask = cv2.bitwise_or(black_mask, white_mask)
        
        # Clean up mask
        kernel = np.ones((5, 5), np.uint8)
        board_mask = cv2.morphologyEx(board_mask, cv2.MORPH_CLOSE, kernel)
        board_mask = cv2.morphologyEx(board_mask, cv2.MORPH_OPEN, kernel)
        
        # Show intermediate masks for debugging
        # cv2.imshow('Black Squares Mask', black_mask)
        # cv2.imshow('White Squares Mask', white_mask)
        # cv2.imshow('Combined Board Mask', board_mask)
        # cv2.waitKey(0)
        
        # Find contours
        contours, _ = cv2.findContours(board_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours
        frame_area = frame.shape[0] * frame.shape[1]
        board_contour = None
        max_area = 0
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < frame_area * 0.2 or area > frame_area * 0.95:
                continue
            
            rect = cv2.minAreaRect(contour)
            width, height = rect[1]
            if width == 0 or height == 0:
                continue
            
            aspect_ratio = max(width, height) / min(width, height)
            # print(aspect_ratio)
            if 0.8 <= aspect_ratio <= 1.2:
                hull = cv2.convexHull(contour)
                solidity = area / cv2.contourArea(hull)

                # print(f"solidity: {solidity}, area: {area}, max_area: {max_area}")

                if solidity > 0.4 and area > max_area:
                    corners = cv2.boxPoints(rect)
                    corners = np.intp(corners)
                    
                    # Contract slightly to fit inside the actual squares
                    center = np.mean(corners, axis=0)
                    corners = corners + (corners - center) * -0.02  # Negative for contraction
                    
                    max_area = area
                    board_contour = corners
        
        if board_contour is None:
            raise ValueError("Could not detect the chess board")
        
        # Sort corners and calculate board dimensions
        board_corners = self.sort_corners(board_contour)
        
        # Calculate board dimensions and make square
        self.board_width = int(max(
            np.linalg.norm(board_corners[0] - board_corners[1]),
            np.linalg.norm(board_corners[2] - board_corners[3])
        ))
        self.board_height = self.board_width  # Force square shape

        # Calculate border padding (assuming border is ~5% of board width)
        border_padding = int(self.board_width * 0.05)

        # Adjust corners to exclude border
        # Move each corner inward by the padding amount
        center = np.mean(board_corners, axis=0)
        adjusted_corners = []
        for corner in board_corners:
            # Calculate vector from center to corner
            vector = corner - center
            # Normalize vector
            vector_norm = np.linalg.norm(vector)
            if vector_norm > 0:
                unit_vector = vector / vector_norm
                # Move corner inward by padding amount
                adjusted_corner = corner - (unit_vector * border_padding)
                adjusted_corners.append(adjusted_corner)
        
        board_corners = np.array(adjusted_corners)
        
        # Update dimensions after padding
        self.board_width = int(max(
            np.linalg.norm(board_corners[0] - board_corners[1]),
            np.linalg.norm(board_corners[2] - board_corners[3])
        ))
        self.board_height = self.board_width
        
        # Destination points for perspective transform
        dst_size = 800
        dst_corners = np.array([
            [0, 0],
            [dst_size - 1, 0],
            [dst_size - 1, dst_size - 1],
            [0, dst_size - 1]
        ], dtype=np.float32)
        
        # Perspective transform matrices
        self.transform_matrix = cv2.getPerspectiveTransform(
            board_corners.astype(np.float32),
            dst_corners
        )
        self.inverse_transform_matrix = cv2.getPerspectiveTransform(
            dst_corners,
            board_corners.astype(np.float32)
        )
        
        # Store board coordinates
        self.board_coords = (
            tuple(board_corners[0].astype(int)),
            tuple(board_corners[2].astype(int))
        )
        
        # Calculate cell sizes
        self.cell_width = self.board_width / 8
        self.cell_height = self.board_height / 8
        
        # Debug visualization
        debug_frame = frame.copy()
        board_outline = np.array([board_corners], dtype=np.int32)
        cv2.polylines(debug_frame, [board_outline], True, (0, 255, 0), 2)
        
        # Draw grid lines
        for i in range(9):
            start_point = self.interpolate_point(board_corners[0], board_corners[3], i/8)
            end_point = self.interpolate_point(board_corners[1], board_corners[2], i/8)
            cv2.line(debug_frame, tuple(map(int, start_point)), tuple(map(int, end_point)), (0, 255, 0), 1)
            
            start_point = self.interpolate_point(board_corners[0], board_corners[1], i/8)
            end_point = self.interpolate_point(board_corners[3], board_corners[2], i/8)
            cv2.line(debug_frame, tuple(map(int, start_point)), tuple(map(int, end_point)), (0, 255, 0), 1)
        
        cv2.imshow('Board Detection Debug', debug_frame)
        cv2.waitKey(1)

    @staticmethod
    def interpolate_point(p1, p2, ratio):
        """Interpolate between two points."""
        return p1 + (p2 - p1) * ratio

    @staticmethod
    def sort_corners(corners: np.ndarray) -> np.ndarray:
        """Sort corners in clockwise order: top-left, top-right, bottom-right, bottom-left."""
        # Calculate the center of the board
        center = corners.mean(axis=0)
        
        # Calculate angles from center
        angles = np.arctan2(corners[:, 1] - center[1],
                        corners[:, 0] - center[0])
        
        # Sort corners based on angles and adjust for correct orientation
        sorted_corners = corners[np.argsort(angles)]
        
        # Rotate so that top-left corner comes first
        # Top-left corner should have smallest sum of x and y coordinates
        corner_sums = sorted_corners.sum(axis=1)
        rotation = -np.argmin(corner_sums)
        sorted_corners = np.roll(sorted_corners, rotation, axis=0)
        
        return sorted_corners

    def get_square_notation(self, x: float, y: float) -> Optional[str]:
        """Convert image coordinates to chess square notation with improved accuracy."""
        # Create homogeneous coordinates
        point = np.array([[[x, y]]], dtype=np.float32)
        
        # Apply perspective transform
        transformed = cv2.perspectiveTransform(point, self.transform_matrix)
        nx, ny = transformed[0][0]
        
        # Calculate square indices
        file_idx = 7 - int((nx / 800) * 8)  # Reverse file index
        # rank_idx = int((ny / 800) * 8)  # Normal rank index
        # file_idx = int((nx / 800) * 8)
        rank_idx = 7 - int((ny / 800) * 8)  # Invert y-axis for chess notation
        
        # Validate indices
        if not (0 <= file_idx < 8 and 0 <= rank_idx < 8):
            return None
        
        # Convert to algebraic notation
        file = chr(ord('a') + file_idx)
        rank = str(8 - rank_idx)  # Reverse rank number
        # rank = str(rank_idx + 1)
        
        return file + rank
    
    def save_pgn(self, moves: List[ChessMove], filename: str = None):
        """
        Save the game in PGN format.
        Automatically detects if game starts with black based on the first move's piece color.
        """
        if not moves:
            raise ValueError("No moves provided")
        
        # Auto-detect if game starts with black by checking first piece color
        starts_with_black = moves[0].piece.startswith('black-')
        
        if filename is None:
            filename = f"game_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pgn"
            
        with open(filename, 'w') as f:
            # Convert moves to algebraic notation
            algebraic_moves = [self.convert_to_algebraic(m) for m in moves]

            # Write moves
            move_text = []

            if starts_with_black:
                # Handle first black move specially
                move_text.append(f"1... {algebraic_moves[0]}")
                start_idx = 1
                move_num = 2
            else:
                start_idx = 0
                move_num = 1

            # Process remaining moves
            i = start_idx
            while i < len(algebraic_moves):
                current_move = []
                
                # Add move number at start of white's move
                if i % 2 == (0 if not starts_with_black else 1):
                    current_move.append(f"{move_num}.")
                    
                # Add the move
                current_move.append(algebraic_moves[i])
                
                # If we have a black move and it's not the last move
                if i + 1 < len(algebraic_moves):
                    current_move.append(algebraic_moves[i + 1])
                    i += 2
                    move_num += 1
                else:
                    i += 1
                    move_num += 1
                
                move_text.append(" ".join(current_move))
            
            # Write all moves with proper spacing
            f.write(" ".join(move_text))
        
        return " ".join(move_text)
    
    def detect_pieces(self, frame: np.ndarray) -> Dict[str, str]:
        """Detect chess pieces and their positions on the board."""
        results = self.vision_system.model(frame, verbose=False)[0]
        positions = {}
        
        for r in results.boxes.data.tolist():
            x1, y1, x2, y2, conf, cls = r
            if conf < self.confidence_threshold:
                continue
            
            # Calculate center point
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            
            # Get square notation
            square = self.get_square_notation(center_x, center_y) 
            if square:
                piece_type = self.vision_system.piece_classes[int(cls)]
                # current_detections.append((square, piece_type))
                positions[square] = piece_type
                
        return positions
    
    def to_board(self, positions: Dict[str, str]) -> chess.Board:
        """Convert board positions chess.Board object."""
        piece_fen_symbols = {
            'white-pawn': 'P', 'black-pawn': 'p',
            'white-knight': 'N', 'black-knight': 'n',
            'white-bishop': 'B', 'black-bishop': 'b',
            'white-rook': 'R', 'black-rook': 'r',
            'white-queen': 'Q', 'black-queen': 'q',
            'white-king': 'K', 'black-king': 'k'
        }
        board = chess.Board()
        board.clear_board()
        
        for square, piece in positions.items():
            board.set_piece_at(chess.parse_square(square), chess.Piece.from_symbol(piece_fen_symbols[piece]))
        
        return board

    def detect_move(self, prev_positions: Dict[str, str], curr_positions: Dict[str, str]) -> Optional[List[ChessMove]]:
        """Detect chess moves by comparing two board positions."""
        if not prev_positions or not curr_positions:
            return None

        vanished_pieces = set(prev_positions.items()) - set(curr_positions.items())
        appeared_pieces = set(curr_positions.items()) - set(prev_positions.items())

        print(f"Found {vanished_pieces}, {appeared_pieces}")
        
        if not vanished_pieces or not appeared_pieces:
            return None
        
        curr_board = self.to_board(curr_positions)

        # Find pieces that moved
        moves = []

        if len(vanished_pieces) == 2 and len(appeared_pieces) == 2: 
            # Castling
            print('Castling')
            if ('e1', 'white-king') in vanished_pieces and ('h1', 'white-rook') in vanished_pieces:
                # O-O
                return [ChessMove('white-king', 'e1', 'g1', False, False, False), 
                        ChessMove('white-rook', 'h1', 'f1', False, False, False)]
            elif ('e1', 'white-king') in vanished_pieces and ('a1', 'white-rook') in vanished_pieces:
                # O-O-O
                return [ChessMove('white-king', 'e1', 'c1', False, False, False), 
                        ChessMove('white-rook', 'a1', 'd1', False, False, False)]
            elif ('e8', 'black-king') in vanished_pieces and ('h8', 'black-rook') in vanished_pieces:
                # O-O
                return [ChessMove('black-king', 'e8', 'g8', False, False, False), 
                        ChessMove('black-rook', 'h8', 'f8', False, False, False)]
            elif ('e8', 'black-king') in vanished_pieces and ('a8', 'black-rook') in vanished_pieces:
                # O-O-O
                return [ChessMove('black-king', 'e8', 'c8', False, False, False), 
                        ChessMove('black-rook', 'a8', 'd8', False, False, False)]
            
        elif len(vanished_pieces) == 2 and len(appeared_pieces) == 1:
            # Capturing
            print('Capturing')
            for (vsquare, vpiece) in vanished_pieces:
                for (asquare, apiece) in appeared_pieces:
                    if vsquare != asquare:
                        to_move = 'b' if apiece.startswith('white') else 'w'
                        fen = curr_board.fen()
                        fen = fen.replace('w', to_move)
                        curr_board.set_fen(fen)
                        return [ChessMove(
                            piece=apiece,
                            from_square=vsquare,
                            to_square=asquare,
                            is_capture=True,
                            is_check=curr_board.is_check(),
                            is_checkmate=curr_board.is_checkmate()
                        )]
        
        elif len(vanished_pieces) == 1 and len(appeared_pieces) == 1:
            # Normal move or promotion
            print('Normal move or promotion')
            to_move = 'b' if list(appeared_pieces)[0][1].startswith('white') else 'w'
            fen = curr_board.fen()
            fen = fen.replace('w', to_move)
            curr_board.set_fen(fen)
            return [ChessMove(
                piece=list(appeared_pieces)[0][1],
                from_square=list(vanished_pieces)[0][0],
                to_square=list(appeared_pieces)[0][0],
                is_capture=False,
                is_check=curr_board.is_check(),
                is_checkmate=curr_board.is_checkmate()
            )]
        else:
            # Invalid move
            print('Invalid move')
            return None

    @staticmethod
    def is_position_different(pos1: Dict[str, str], pos2: Dict[str, str], threshold: float = 1/33) -> bool:
        """Check if positions are significantly different."""
        if not pos1 or not pos2:
            return True
            
        differences = sum(1 for k, v in pos1.items() if k not in pos2 or pos2[k] != v)
        total = len(set(pos1.keys()) | set(pos2.keys())) # len union of squares
        
        return differences / total >= threshold

    @staticmethod
    def is_position_similar(pos1: Dict[str, str], pos2: Dict[str, str], threshold: float = 32/33) -> bool:
        """Check if positions are similar enough to be considered the same."""
        if not pos1 or not pos2:
            return False
            
        common = sum(1 for k, v in pos1.items() if k in pos2 and pos2[k] == v)
        total = len(set(pos1.keys()) | set(pos2.keys())) # len union of squares
        
        return common / total >= threshold

    @staticmethod
    def convert_to_algebraic(move: ChessMove) -> str:
        """Convert move to algebraic notation."""
        piece_symbols = {
            'white-pawn': '', 'black-pawn': '',
            'white-knight': 'N', 'black-knight': 'N',
            'white-bishop': 'B', 'black-bishop': 'B',
            'white-rook': 'R', 'black-rook': 'R',
            'white-queen': 'Q', 'black-queen': 'Q',
            'white-king': 'K', 'black-king': 'K'
        }
        
        notation = []
        
        # Add piece symbol
        notation.append(piece_symbols[move.piece])
        
        # Add capture symbol if needed
        if move.is_capture:
            if not notation[0]:  # Pawn capture
                notation.append(move.from_square[0])
            notation.append('x')
            
        # Add destination square
        notation.append(move.to_square)
        
        # Add check/checkmate symbol
        if move.is_checkmate:
            notation.append('#')
        elif move.is_check:
            notation.append('+')
            
        return ''.join(notation)

    def display_analysis(self, frame: np.ndarray, curr_positions: Dict[str, str], moves: List[ChessMove]) -> None:
        """Display real-time analysis with detected pieces and moves."""
        # Create a copy of frame to draw on
        display = frame.copy()
        
        # Resize frame to fit in 400x400 while maintaining aspect ratio
        height, width = display.shape[:2]
        max_size = 900
        if height > max_size or width > max_size:
            scale = max_size / float(max(height, width))
            new_size = (int(width * scale), int(height * scale))
            display = cv2.resize(display, new_size)
        
        # Update coordinates based on resize
        scale_x = display.shape[1] / frame.shape[1]
        scale_y = display.shape[0] / frame.shape[0]
        board_top_left = (int(self.board_coords[0][0] * scale_x), int(self.board_coords[0][1] * scale_y))
        board_bottom_right = (int(self.board_coords[1][0] * scale_x), int(self.board_coords[1][1] * scale_y))
        
        # Draw full board border
        cv2.rectangle(display, board_top_left, board_bottom_right, (0, 255, 0), 2)
        
        # Draw grid lines
        board_width = board_bottom_right[0] - board_top_left[0]
        board_height = board_bottom_right[1] - board_top_left[1]
        cell_width = board_width / 8
        cell_height = board_height / 8
        
        # Draw vertical grid lines
        for i in range(1, 8):
            x = int(board_top_left[0] + i * cell_width)
            cv2.line(display, 
                    (x, board_top_left[1]), 
                    (x, board_bottom_right[1]), 
                    (0, 255, 0), 1)
        
        # Draw horizontal grid lines
        for i in range(1, 8):
            y = int(board_top_left[1] + i * cell_height)
            cv2.line(display, 
                    (board_top_left[0], y), 
                    (board_bottom_right[0], y), 
                    (0, 255, 0), 1)
        
        # Draw detected pieces
        for square, piece in curr_positions.items():
            # Debugging
            # print(f"square: {square}, piece: {piece}")

            # Calculate cell center
            # file_idx = ord(square[0]) - ord('a')
            # rank_idx = 7 - (int(square[1]) - 1)
            file_idx = 7 - (ord(square[0]) - ord('a'))  # Reverse file index
            rank_idx = int(square[1]) - 1  # Normal rank index
            
            cell_x = int(board_top_left[0] + (file_idx + 0.5) * cell_width)
            cell_y = int(board_top_left[1] + (rank_idx + 0.5) * cell_height)
            
            # Draw piece marker
            cv2.circle(display, (cell_x, cell_y), 3, (0, 0, 255), -1)
            
            # Draw piece label
            piece_label = piece.split('-')[1][:2]  # First two letters of piece name
            color = (0, 0, 255) if piece.startswith('black') else (255, 0, 0)
            cv2.putText(display, piece_label, 
                    (cell_x - 15, cell_y - 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        # Create sidebar for move list
        sidebar_width = 250
        full_height = display.shape[0]
        full_width = display.shape[1] + sidebar_width
        
        # Create white background
        full_display = np.ones((full_height, full_width, 3), dtype=np.uint8) * 255
        
        # Copy the resized frame
        full_display[:display.shape[0], :display.shape[1]] = display
        
        # Draw separator line
        cv2.line(full_display, 
                (display.shape[1], 0), 
                (display.shape[1], full_height), 
                (0, 0, 0), 1)
        
        # Add title
        cv2.putText(full_display, "Detected Moves:", 
                (display.shape[1] + 10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        # Display moves
        for i, move in enumerate(moves[-12:]):  # Show last 12 moves
            move_text = f"{len(moves)-11+i}. {self.convert_to_algebraic(move)}"
            cv2.putText(full_display, move_text,
                    (display.shape[1] + 10, 50 + i*20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        # Display state information
        state_color = (0, 255, 0) if self.state == 'STABLE' else (0, 0, 255)
        cv2.putText(full_display, f"State: {self.state}",
                (display.shape[1] + 10, full_height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, state_color, 1)
        
        # Show the analysis
        cv2.imshow('Chess Analysis', full_display)

    def analyze_video(self, display: bool = True) -> List[str]:
        """Analyze chess video and return moves in algebraic notation."""
        from IPython.display import clear_output
        
        moves = []
        frame_count = 0
        skip_frames = 2

        # Motion detection parameters
        motion_threshold = 15  # Minimum pixel difference to detect motion
        motion_area_threshold = 0.001  # Minimum percentage of changed pixels to detect significant motion
        kernel = np.ones((5,5), np.uint8)

        ret, prev_frame = self.cap.read()
        if not ret:
            return moves
        
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # skip frame logic (includes the first frame)
            frame_count += 1
            if frame_count % skip_frames != 0: 
                continue

            # Hand detection
            predicted, org, segmented, count = hand_is_in_frame(frame)
            if predicted:
                continue

            # Motion detection
            curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_diff = cv2.absdiff(prev_gray, curr_gray)

            # Threshold and clean up motion mask
            motion_mask = cv2.threshold(frame_diff, motion_threshold, 255, cv2.THRESH_BINARY)[1]
            motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel)
            motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel)

            # Calculate percentage of pixels showing motion
            motion_percentage = np.sum(motion_mask > 0) / (motion_mask.shape[0] * motion_mask.shape[1])
        
            if motion_percentage > motion_area_threshold:
                prev_gray = curr_gray.copy()
                continue

            # Detect pieces
            curr_positions = self.detect_pieces(frame)
            
            # Clear output and display board state every 10 frames
            # if frame_count % 10 == 0:
            #     clear_output(wait=True)
            #     print(f"\nFrame {frame_count}")
            #     self.print_board_state(curr_positions)
            
            if not self.prev_stable_positions:
                self.prev_stable_positions = curr_positions.copy()

            # State machine for move detection
            if self.state == 'STABLE':
                if self.is_position_different(self.prev_stable_positions, curr_positions):
                    self.state = 'UNSTABLE'
                    self.stable_frame_count = 0
            else: # UNSTABLE
                if self.is_position_similar(curr_positions, self.prev_positions):
                    self.stable_frame_count += 1
                    if self.stable_frame_count >= self.stable_threshold:
                        # Detected a stable position
                        self.state = 'STABLE'
                        # Register a move
                        move = self.detect_move(self.prev_stable_positions, curr_positions)
                        # print(f"Move: {move}")
                        if move:
                            moves += move
                        self.prev_stable_positions = curr_positions.copy()
                else:
                    self.stable_frame_count = 0
                    
            self.prev_positions = curr_positions.copy()
            prev_gray = curr_gray.copy()
            
            if display:
                self.display_analysis(frame, curr_positions, moves)
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        self.cap.release()
        cv2.destroyAllWindows()
        
        # return [self.convert_to_algebraic(m) for m in moves]
        return moves

    def print_board_state(self, positions: Dict[str, str]) -> None:
        """Print current board state in a chess-like format."""
        # Create empty board
        board = [['.'] * 8 for _ in range(8)]
        
        # Define piece symbols
        piece_symbols = {
            'white_pawn': 'P', 'black_pawn': 'p',
            'white_knight': 'N', 'black_knight': 'n',
            'white_bishop': 'B', 'black_bishop': 'b',
            'white_rook': 'R', 'black_rook': 'r',
            'white_queen': 'Q', 'black_queen': 'q',
            'white_king': 'K', 'black_king': 'k'
        }
        
        # Fill board with detected pieces
        for square, piece in positions.items():
            # file_idx = ord(square[0]) - ord('a')
            rank_idx = 8 - int(square[1])
            file_idx = 7 - (ord(square[0]) - ord('a'))  # Reverse file index
            # rank_idx = int(square[1]) - 1
            board[rank_idx][file_idx] = piece_symbols.get(piece, '?')
        
        # Print board with coordinates
        # print("\n  a b c d e f g h")
        # print("  ─────────────────")
        # for i, row in enumerate(board):
        #     print(f"{8-i}│{' '.join(row)}│{8-i}")
        # print("  ─────────────────")
        # print("  a b c d e f g h")
        print("\n  h g f e d c b a")  # Reversed files
        print("  ─────────────────")
        for i, row in enumerate(board):
            print(f"{i+1}│{' '.join(row)}│{i+1}")  # Normal ranks
        print("  ─────────────────")
        print("  h g f e d c b a")  # Reversed files
        
        # Print current state
        print(f"\nState: {self.state}")
    


## Training the Model

Let's create a function to train the chess piece detection model.

In [77]:
def train_model(data_yaml_path: str, epochs: int = 100):
    """Train the chess piece detection model."""
    print(f"Starting training with {epochs} epochs...")
    vision_system = ChessVisionSystem()
    
    try:
        vision_system.train(
            data_yaml_path=data_yaml_path,
            epochs=epochs
        )
        print("Training complete!")
    except Exception as e:
        print(f"Error during training: {str(e)}")
        raise

In [78]:
# Training example
DATA_YAML_PATH = './custom_datasets/data.yaml'  # Update this path
EPOCHS = 100

# Uncomment to train the model
# train_model(DATA_YAML_PATH, EPOCHS)

## Analyzing Chess Games

Now let's create functions for analyzing chess games from video.

In [79]:
def analyze_game(model_path: str, video_path: str, output_path: str = 'analyzed_game.pgn', rotate: bool = False):
    """Analyze a chess game from video."""
    print("Starting chess game analysis...")
    
    try:
        # Read the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")
            
        # Read first frame to check if we need to rotate
        ret, frame = cap.read()
        if not ret:
            raise ValueError("Could not read first frame")
            
        if rotate:
            # Rotate 90 degrees clockwise
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            
        # Release and reopen video to start from beginning
        cap.release()

        # Create a temporary rotated video if needed
        if rotate:
            temp_video_path = video_path.replace('.mp4', '_rotated.mp4')
            
            # Get original video properties
            original_cap = cv2.VideoCapture(video_path)
            fps = original_cap.get(cv2.CAP_PROP_FPS)
            
            # Create video writer
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            height, width = frame.shape[:2]
            out = cv2.VideoWriter(temp_video_path, fourcc, fps, (width, height))
            
            # Process each frame
            while True:
                ret, frame = original_cap.read()
                if not ret:
                    break
                    
                # Rotate frame
                rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
                out.write(rotated_frame)
                
            # Release resources
            original_cap.release()
            out.release()
            
            # Use the rotated video for analysis
            video_path = temp_video_path

        analyzer = ChessGameAnalyzer(
            model_path=model_path,
            video_path=video_path
        )
        
        moves = analyzer.analyze_video(display=True)
        
        # Save the game
        moves_result = analyzer.save_pgn(moves, output_path)
        print(f"Analysis complete. Detected {len(moves)} moves.")
        print(f"Game saved to '{output_path}'")
        
        return moves_result
    
    except Exception as e:
        print(f"Error during analysis: {str(e)}")
        raise
    finally:
        cv2.destroyAllWindows()

## Example Usage

Now let's analyze a chess game video:

In [80]:
# # Analysis example
# MODEL_PATH = './runs/detect/ming_runs/weights/best.pt'  # Update this path
# VIDEO_PATH = './data/test_videos/2_move_student.mp4'  # Update this path
# OUTPUT_PATH = '2_move_student.pgn'

# # Analyze the game
# moves_result = analyze_game(MODEL_PATH, VIDEO_PATH, OUTPUT_PATH)

## Predict Result

In [81]:
def export_move_to_csv(moves: Dict[str, str], filename: str):
  """
    Export moves to a CSV file with columns 'row_id' and 'output'.
    
    Parameters:
        moves: Dictionary of row_id to move string
        filename: Output CSV filename
    
    Example moves dict:
    {
        "1": "1. e4 e5",
        "2": "2. Nf3 Nc6",
        "3": "3. Bb5 a6"
    }
  """
  import pandas as pd
    
  # Convert dictionary to DataFrame
  df = pd.DataFrame(list(moves.items()), columns=['row_id', 'output'])
  
  # Add .csv extension if not present
  if not filename.endswith('.csv'):
      filename = filename + '.csv'
  
  # Export to CSV without index
  df.to_csv(filename, index=False)
  
  return df  # Return DataFrame for potential further use

In [82]:
MODEL_PATH = './yolo_model/weights/best.pt'

VIDEO_NAMES = ["2_Move_rotate_student", "2_move_student", "4_Move_studet", "6_Move_student", "8_Move_student"]
# VIDEO_NAMES = ["8_Move_student-Trim"]

results = {}

for video_name in VIDEO_NAMES:
  VIDEO_PATH = f'./data/test_videos/{video_name}.mp4'
  OUTPUT_PATH = f'{video_name}.pgn'

  needs_rotation = 'rotate' in video_name.lower()
  
  moves_result = analyze_game(MODEL_PATH, VIDEO_PATH, OUTPUT_PATH, rotate=needs_rotation)
  results[f"{video_name}.mp4"] = moves_result

export_move_to_csv(results, "submission")

Starting chess game analysis...
Found {('d8', 'black-queen')}, {('h4', 'black-queen')}
Normal move or promotion
Found {('g2', 'white-pawn')}, {('g3', 'white-pawn')}
Normal move or promotion
Analysis complete. Detected 2 moves.
Game saved to '2_Move_rotate_student.pgn'
Starting chess game analysis...
Found {('d8', 'black-queen')}, {('h4', 'black-queen')}
Normal move or promotion
Found {('g2', 'white-pawn')}, {('g3', 'white-pawn')}
Normal move or promotion
Analysis complete. Detected 2 moves.
Game saved to '2_move_student.pgn'
Starting chess game analysis...
Found {('f6', 'black-pawn')}, {('f4', 'black-pawn')}
Normal move or promotion
Found {('h4', 'white-knight')}, {('g6', 'white-knight')}
Normal move or promotion
Found {('g3', 'white-bishop'), ('f4', 'black-pawn')}, {('g3', 'black-pawn')}
Capturing
Found {('h8', 'black-rook'), ('g6', 'white-knight')}, {('h8', 'white-knight')}
Capturing
Analysis complete. Detected 4 moves.
Game saved to '4_Move_studet.pgn'
Starting chess game analysis..

,row_id,output
0,2_Move_rotate_student.mp4,1... Qh4+ 2. g3
1,2_move_student.mp4,1... Qh4+ 2. g3
2,4_Move_studet.mp4,1... f4 2. Ng6 fxg3 3. Nxh8
3,6_Move_student.mp4,1... Bxb5 2. Rxb5 b6 3. c4 Ne7 4. Rb2
4,8_Move_student.mp4,1. Qe6 Kd8 2. Qf7 c6 3. Qf2 cxd5 4. Qxa7 Rc8 5...
